In [8]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import uuid

In [9]:
# Define the input and output directories
input_dir = "../dataset/images"
output_dir = "../dataset/augmented_images"

In [10]:
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [11]:
# Labels for sign language dataset
labels = list("123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ")

# Create subdirectories for augmented images
for label in labels:
    os.makedirs(os.path.join(output_dir, label), exist_ok=True)

In [12]:
# Define the augmentation generator
datagen = ImageDataGenerator(
    rotation_range=30,          # Rotate images up to 30 degrees
    width_shift_range=0.1,      # Horizontal shift
    height_shift_range=0.1,     # Vertical shift
    shear_range=0.2,            # Shear transformation
    zoom_range=0.2,             # Random zoom
    horizontal_flip=True,       # Randomly flip images horizontally
    fill_mode='nearest'         # Fill any empty space after transformations
)

In [13]:
# Function to augment and save images with resizing
def augment_images(input_dir, output_dir, label, augment_count=5):
    input_path = os.path.join(input_dir, label)
    output_path = os.path.join(output_dir, label)

    for img_name in os.listdir(input_path):
        if img_name.endswith(".jpg"):
            img_path = os.path.join(input_path, img_name)
            
            # Load the image
            img = tf.keras.preprocessing.image.load_img(img_path)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            
            # Resize the image to 32x32
            img_array = tf.image.resize(img_array, (300, 300))  # Resize to 32x32
            img_array = img_array.numpy()  # Convert tensor to numpy array
            
            img_array = img_array.reshape((1,) + img_array.shape)  # Reshape for generator
            
            # Generate augmented images
            count = 0
            for batch in datagen.flow(img_array, batch_size=1):
                aug_img = tf.keras.preprocessing.image.array_to_img(batch[0])
                
                # Save the augmented image with a unique name
                aug_filename = os.path.join(output_path, f"{uuid.uuid4().hex}.jpg")
                aug_img.save(aug_filename)
                
                count += 1
                if count >= augment_count:  # Stop after creating specified number of augmentations
                    break

In [14]:
# Apply augmentation for each label
for label in labels:
    augment_images(input_dir, output_dir, label, augment_count=5)

print("Augmentation complete!")

Augmentation complete!
